In [1]:
# Import packages and modules
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df  = pd.read_csv("../Data/dataset_dk3619_preprocessed_v2_dropnan.csv")

In [3]:
df['Consumption_MWh'].head()

0    144.276350
1    231.498108
2     48.562569
3    169.283123
4    131.341630
Name: Consumption_MWh, dtype: float64

In [10]:
 # Boolean indexing
data = df[
            (df['year'] == 2024) & 
            (df['month'] == 5) &  
            (df['day_of_month'] == 30) & 
            (df['DK3619Code'] == 'C_CC')
]


In [8]:
data.to_csv("../Data/test.csv", index=False)

In [4]:
print(df['DK3619Code'])
dkcode_list = list(set(df['DK3619Code'].to_list()))
dkcode_list.sort()
id2dkcode = enumerate(dkcode_list)
id2dkcode = dict(id2dkcode)

dkcode2id = {}
for i in id2dkcode.items():
    dkcode2id[i[1]] = i[0]


print(dict(dkcode2id))

### Mapping data to numberic
# df['DK3619Code'] = df['DK3619Code'].map(dkcode2id)

0                 A_A
1                C_CA
2                C_CC
3          C_CE_CF_CD
4                C_CG
              ...    
3322206          Q_QB
3322207           R_R
3322208           S_S
3322209    C_CE_CF_CD
3322210          M_MA
Name: DK3619Code, Length: 3322211, dtype: object
{'A_A': 0, 'C_CA': 1, 'C_CC': 2, 'C_CE_CF_CD': 3, 'C_CG': 4, 'C_CH': 5, 'C_CI': 6, 'C_CJ': 7, 'C_CK': 8, 'C_CL': 9, 'C_CM_CB': 10, 'D_D': 11, 'E_E': 12, 'F_F': 13, 'G_G': 14, 'H_H': 15, 'I_I': 16, 'J_JA': 17, 'J_JB_JC': 18, 'K_K': 19, 'L_L': 20, 'M_MA': 21, 'M_MB': 22, 'M_MC': 23, 'N_N': 24, 'O_O': 25, 'PR_PR': 26, 'P_P': 27, 'Q_QA': 28, 'Q_QB': 29, 'R_R': 30, 'S_S': 31}


In [22]:
# Export the dkcode_list to a CSV file
dkcode_df = pd.DataFrame(dkcode_list, columns=['DK3619Code'])
dkcode_df.to_csv('dkcode_list.csv', index=False)

### Choose Feauture will use for Model

In [5]:
columns = ['HourUTC', 'HourDK', 'hour', 'day_of_week',
              'weekday_name', 'quarter', 'month', 'year', 'day_of_year',
              'day_of_month', 'week_of_year', 'season', 'holiday', 'Consumption_MWh']


In [6]:
df.columns

Index(['HourUTC', 'HourDK', 'DK3619Code', 'hour', 'day_of_week',
       'weekday_name', 'quarter', 'month', 'year', 'day_of_year',
       'day_of_month', 'week_of_year', 'season', 'holiday', 'DK36Code',
       'DK36Title', 'DK19Code', 'DK19Title', 'Consumption_MWh', 'lag_24',
       'lag_25', 'lag_26', 'lag_27', 'lag_28', 'lag_29', 'lag_30', 'lag_31',
       'lag_32', 'lag_33', 'lag_34', 'lag_35', 'lag_36', 'lag_37', 'lag_38',
       'lag_39', 'lag_40', 'lag_41', 'lag_42', 'lag_43', 'lag_44', 'lag_45',
       'lag_46', 'lag_47', 'lag_48'],
      dtype='object')

In [8]:
# print(df['weekday_name'])
# weekday_list = list(set(df['weekday_name'].to_list()))
# weekday_list.sort()
# id2day = enumerate(weekday_list)
# id2day = dict(id2day)

# day2id = {}
# for i in id2day.items():
#     day2id[i[1]] = i[0]
# print(dict(day2id))

### Mapping data to numberic
# df['weekday_name'] = df['weekday_name'].map(day2id)

In [7]:
weekday_mapping = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}
### Mapping data to numberic
df['weekday_name'] = df['weekday_name'].map(weekday_mapping)

In [10]:
# print(df['season'])
# season_list = list(set(df['season'].to_list()))
# season_list.sort()
# id2season = enumerate(season_list)
# id2season = dict(id2season)

# season2id = {}
# for i in id2season.items():
#     season2id[i[1]] = i[0]
# print(dict(season2id))

### Season mapping
# df['season'] = df['season'].map(season2id)


In [8]:

### Season mapping

seasons_mapping = {
    'Spring': 0,
    'Summer': 1,
    'Autumn': 2,
    'Winter': 3
}

df['season'] = df['season'].map(seasons_mapping)

In [9]:
df_corr = df[columns]
df_corr.head()

,HourUTC,HourDK,hour,day_of_week,weekday_name,quarter,month,year,day_of_year,day_of_month,week_of_year,season,holiday,Consumption_MWh
0,2012-06-01T23:00:00,2012-06-02 01:00:00,1,5,5,2,6,2012,154,2,22,1,0,0.068992
1,2012-06-01T23:00:00,2012-06-02 01:00:00,1,5,5,2,6,2012,154,2,22,1,0,8.836999
2,2012-06-02T00:00:00,2012-06-02 02:00:00,2,5,5,2,6,2012,154,2,22,1,0,74.687279
3,2012-06-02T00:00:00,2012-06-02 02:00:00,2,5,5,2,6,2012,154,2,22,1,0,0.565844
4,2012-06-02T00:00:00,2012-06-02 02:00:00,2,5,5,2,6,2012,154,2,22,1,0,22.456516


In [13]:
df.head()

,HourUTC,HourDK,DK3619Code,hour,day_of_week,weekday_name,quarter,month,year,day_of_year,...,lag_39,lag_40,lag_41,lag_42,lag_43,lag_44,lag_45,lag_46,lag_47,lag_48
0,2020-01-01T00:00:00,2020-01-01 01:00:00,R_R,1,2,2,1,1,2020,1,...,39.452003,37.029076,34.272778,30.867796,27.691839,24.262220,21.446964,20.692560,20.760229,21.076712
1,2020-01-01T00:00:00,2020-01-01 01:00:00,Q_QB,1,2,2,1,1,2020,1,...,13.973817,13.383460,12.440164,11.175482,9.128553,7.690116,7.325366,7.297887,7.436323,7.600429
2,2020-01-01T00:00:00,2020-01-01 01:00:00,Q_QA,1,2,2,1,1,2020,1,...,9.143302,8.965540,8.633377,6.069151,3.955089,3.400593,3.144497,3.085907,3.090541,3.118976
3,2020-01-01T00:00:00,2020-01-01 01:00:00,PR_PR,1,2,2,1,1,2020,1,...,1208.678223,1180.071045,1051.362549,880.705933,789.150513,721.568359,703.513916,712.612061,736.379395,784.044617
4,2020-01-01T00:00:00,2020-01-01 01:00:00,P_P,1,2,2,1,1,2020,1,...,53.242626,53.199566,53.141167,50.922737,46.150143,42.539543,41.061405,40.467297,40.414558,40.375416


### Split data

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dense, Bidirectional
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import TimeDistributed, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

import os.path


In [11]:
print(tf.__version__)
import keras
print(keras.__version__)

2.13.0
2.13.1


In [12]:
# Create sequences
def create_sequences(df, seq_length):
    sequences = []
    labels = []
    for i in range(len(df) - seq_length):
        seq = df.iloc[i:i+seq_length].values
        label = df['Consumption_MWh'].iloc[i+seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [14]:
df_filter = df[df['DK3619Code'] == 'A_A']
df_corr = df_filter[columns]

df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])

df_corr = df_corr.drop(['HourUTC', 'HourDK'], axis=1)

/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_4593/758837340.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_4593/758837340.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])


In [15]:
df_corr

,hour,day_of_week,weekday_name,quarter,month,year,day_of_year,day_of_month,week_of_year,season,holiday,Consumption_MWh
21,2,5,5,2,6,2012,154,2,22,1,0,14.979990
60,3,5,5,2,6,2012,154,2,22,1,0,16.318953
81,4,5,5,2,6,2012,154,2,22,1,0,17.724710
123,5,5,5,2,6,2012,154,2,22,1,0,18.121771
153,6,5,5,2,6,2012,154,2,22,1,0,16.771591
...,...,...,...,...,...,...,...,...,...,...,...,...
3323616,19,4,4,2,6,2024,173,21,25,1,0,173.246406
3323642,20,4,4,2,6,2024,173,21,25,1,0,168.106920
3323679,21,4,4,2,6,2024,173,21,25,1,0,161.731316
3323714,22,4,4,2,6,2024,173,21,25,1,0,153.871964


In [16]:
df_corr[['season', 'year', 'month', 'day_of_month', 'hour', 'Consumption_MWh']]

,season,year,month,day_of_month,hour,Consumption_MWh
21,1,2012,6,2,2,14.979990
60,1,2012,6,2,3,16.318953
81,1,2012,6,2,4,17.724710
123,1,2012,6,2,5,18.121771
153,1,2012,6,2,6,16.771591
...,...,...,...,...,...,...
3323616,1,2024,6,21,19,173.246406
3323642,1,2024,6,21,20,168.106920
3323679,1,2024,6,21,21,161.731316
3323714,1,2024,6,21,22,153.871964


In [17]:
# Assuming df_corr is your DataFrame
df_corr[['season', 'year', 'month', 'day_of_month', 'hour', 'Consumption_MWh']].to_csv('output.csv', index=False)


In [20]:
#all_dkcode_trainings
if not os.path.exists("../Checkpoints/"):
    os.makedirs("../Checkpoints/")

for i in dkcode_list:
    if os.path.exists("../Checkpoints/" + i + "_LSTMmodel.h5"):
        continue
    print('Training LSTM model with DKCode: ', i)
    df_filter = df[df['DK3619Code'] == i]
    df_corr = df_filter[columns]

    df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
    df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])

    df_corr = df_corr.drop(['HourUTC', 'HourDK'], axis=1)

    # Normalize the data
    scaler = MinMaxScaler()
    scaled_df = scaler.fit_transform(df_corr[['season', 'year', 'month', 'day_of_month', 'hour', 'Consumption_MWh']])

    scaled_df = pd.DataFrame(scaled_df, columns=['season', 'year', 'month', 'day_of_month', 'hour', 'Consumption_MWh'])

    #create sequence for training
    SEQ_LENGTH = 24  # for 24 hours sequence
    sequences, labels = create_sequences(scaled_df, SEQ_LENGTH)

    # Split the data into training (60%), validation (20%), and test (20%) sets
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, shuffle=False)

    #LSTM modeling
    #SEQ_LENGTH=24 and  X_train.shape[2] = 6
    model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, X_train.shape[2])),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

    # Model Training
    history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

    model.save("../Checkpoints/" + i + "_LSTMmodel.h5")
    model = tf.keras.models.load_model("../Checkpoints/" + i + "_LSTMmodel.h5")


    # Model Evaluation
    predictions = model.predict(X_test)

    # Calculate evaluation metrics
    r2 = r2_score(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)

    print(f'R² Score: {r2}')
    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')

Training LSTM model with DKCode:  C_CG


<ipython-input-20-c435cd77fc11>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
<ipython-input-20-c435cd77fc11>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])


Epoch 1/10
1568/1568 [==============================] - 20s 10ms/step - loss: 0.0077 - val_loss: 0.0021 - lr: 0.0010
Epoch 2/10
1568/1568 [==============================] - 13s 8ms/step - loss: 0.0032 - val_loss: 0.0015 - lr: 0.0010
Epoch 3/10
1568/1568 [==============================] - 14s 9ms/step - loss: 0.0021 - val_loss: 0.0012 - lr: 0.0010
Epoch 4/10
1568/1568 [==============================] - 13s 8ms/step - loss: 0.0015 - val_loss: 8.5394e-04 - lr: 0.0010
Epoch 5/10
1568/1568 [==============================] - 13s 8ms/step - loss: 0.0011 - val_loss: 5.2561e-04 - lr: 0.0010
Epoch 6/10
1568/1568 [==============================] - 13s 8ms/step - loss: 8.3841e-04 - val_loss: 4.2800e-04 - lr: 0.0010
Epoch 7/10
1568/1568 [==============================] - 14s 9ms/step - loss: 7.6925e-04 - val_loss: 4.7338e-04 - lr: 0.0010
Epoch 8/10
1568/1568 [==============================] - 14s 9ms/step - loss: 7.2223e-04 - val_loss: 3.8312e-04 - lr: 0.0010
Epoch 9/10
1568/1568 [=================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


245/245 [==============================] - 1s 3ms/step
R² Score: 0.9815306466597061
Mean Absolute Error (MAE): 0.018321539687174145
Mean Squared Error (MSE): 0.0004896690900374253
Root Mean Squared Error (RMSE): 0.022128467864663052
Training LSTM model with DKCode:  C_CH


...

245/245 [==============================] - 2s 4ms/step
R² Score: 0.9917566069023023
Mean Absolute Error (MAE): 0.012984516054329038
Mean Squared Error (MSE): 0.0002724623923336379
Root Mean Squared Error (RMSE): 0.016506434876545507
Training LSTM model with DKCode:  S_S


<ipython-input-20-c435cd77fc11>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
<ipython-input-20-c435cd77fc11>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])


Epoch 1/10
1568/1568 [==============================] - 17s 9ms/step - loss: 0.0046 - val_loss: 0.0018 - lr: 0.0010
Epoch 2/10
1568/1568 [==============================] - 13s 8ms/step - loss: 0.0016 - val_loss: 5.3211e-04 - lr: 0.0010
Epoch 3/10
1568/1568 [==============================] - 13s 8ms/step - loss: 8.8999e-04 - val_loss: 3.2804e-04 - lr: 0.0010
Epoch 4/10
1568/1568 [==============================] - 13s 8ms/step - loss: 6.5110e-04 - val_loss: 2.7387e-04 - lr: 0.0010
Epoch 5/10
1568/1568 [==============================] - 12s 8ms/step - loss: 5.4985e-04 - val_loss: 2.7681e-04 - lr: 0.0010
Epoch 6/10
1568/1568 [==============================] - 12s 8ms/step - loss: 4.7916e-04 - val_loss: 2.1781e-04 - lr: 0.0010
Epoch 7/10
1568/1568 [==============================] - 13s 8ms/step - loss: 4.5441e-04 - val_loss: 1.5422e-04 - lr: 0.0010
Epoch 8/10
1568/1568 [==============================] - 13s 8ms/step - loss: 4.1652e-04 - val_loss: 1.7092e-04 - lr: 0.0010
Epoch 9/10
1568/1568

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


245/245 [==============================] - 1s 3ms/step
R² Score: 0.9920413399551986
Mean Absolute Error (MAE): 0.010684802334196363
Mean Squared Error (MSE): 0.00018080916031186929
Root Mean Squared Error (RMSE): 0.013446529675416973
